In [ ]:
%pip install llama-cpp-python fire

In [1]:
import fire
from llama_cpp import Llama
import json

In [22]:
with open("data/categories.json") as f:
    categories = json.loads(f.read())
    categories = list(categories.keys())

text = ""
SYSTEM_PROMPT = "Отнеси описание данного видео к 1 или 2 категориям. Если в описании есть имя собственное добавь категорию Массовая культура. \n" \
        "{categories}\n\n" \
        "Текст: Артмеханика. Концерт Сэм Пикар. Концерт Сэм Пикар. \n" \
        "Ответ: Музыка и аудио, События и достопримечательности. \n\n" \
        "Текст: Злые языки | Выпуск 3, Сезон 4 | Марат Башаров. Магические способности актёра. Скандал с Сафроновым. Опекунство над детьми из Сирии. Новая любовь - скоро свадьба? \n" \
        "Ответ: Массовая культура. \n\n" \
        "Текст: ПО РУКАМ #6. В начале выпуска посмотрим, сможет ли пацанка Настя Петрова победить блогера Влада Левского. ПРО-АРМ: Виктора Шевеленко и Байсангур Солтыханов встретятся в реванше профессионалов. ПОП-АРМ: второй полуфинал! Рекордсмен в подъеме на бицепс Андрей Качалин против фитнес-блогера Алексея Шредера. \n" \
        "Ответ: Спорт, Массовая культура \n\n" \
        "Текст: Пацанский клининг. Шоу «ЧистоТачка» | Повелитель ночи | Выпуск 17. Тяпа и Егор бросили вызов нестареющему «повелителю ночи». Смогут ли они подарить ему статус «ЧистоТачки»? \n" \
        "Ответ: Массовая культура \n\n"

model_path = "data/model-q4_K.gguf"
n_ctx=8192
top_k=30
top_p=0.9
temperature=0.6
repeat_penalty=1.1

messages = [{"role": "system", "content": SYSTEM_PROMPT.format(categories='\n'.join(categories))}]

In [ ]:
import torch
torch.cuda.is_available()

In [54]:
import pandas as pd
df = pd.read_csv("resources/sample_submission.csv")
df

,video_id,title,description,predicted_tags
0,c00d1b173ed8680c155b773d43af7791,Тяпа | #24 | 8 МАРТА. ТЯПА ПОЗДРАВЛЯЕТ ЛИЗУ. З...,Утро Тяпы начинается с недвусмысленного напоми...,NaN
1,6064c4511a067f2e1d1cef81c21421e8,БЮРО ДОБРЫХ ДЕЛ №5 | ХУДОЖНИК ИЗ АСТРАХАНИ | К...,Добро пожаловать на канал “Бюро Добрых Дел”! Е...,NaN
2,3071e2054a02a03150463e514f05fe90,Такой расклад. Эфир 17. Таро. Про личные качес...,В этом выпуске шоу «Такой расклад» вы увидите ...,NaN
3,60867179613376c8b56f0705114a7685,TREXTRAVELER в ГрандТуре «Байкальская миля 202...,"Визит в Государственный природный заказник ""Су...",NaN
4,706f44ef4711eaee99035260c2d1ab86,Тру ДЕТЕКТОР I #5,Вы смотрите шоу с детектором лжи «Тру ДЕТЕКТОР...,NaN
...,...,...,...,...
194,0f84219f46782fca9482fb88da146ed3,Диктант со звездой | Выпуск 7 | Чипинкос,В этом выпуске - рэпер Чипинкос останется один...,NaN
195,2f67917a65dd8d5b348de81d15b1d8ff,Полупановы. «Эпизод 16»,В этом эпизоде Игорь готовится к любимому праз...,NaN
196,6fa39a883591bc0e2cf7ef6c5349c9df,"Салют, Начальник! 2 сезон, 6 серия",Назаров устраивает майору «сюрприз» прямо за с...,NaN
197,bff493ac045715566a86cd7e84051938,Абьюз-шоу |Выпуск №7 часть 1I Алина Егорова,Героиней этого выпуска «Абьюз шоу» стала Алина...,NaN


In [17]:
part_df = df.sample(frac = 0.001)
part_df

,video_id,title,description,tags
913,3de3fdfefe2ccf8ee2a55980ac660a32,Тяпа I #16 I ОЧНУЛСЯ НА НЕОБИТАЕМОМ ОСТРОВЕ. К...,"Подписывайся на все наши каналы на Rutube, нах...",Массовая культура: Юмор и сатира


In [ ]:
# part_df["tags"] = part_df["tags"].apply(lambda x: ", ".join(determine_1_level_categories(x)))
# part_df["tags"]

In [25]:
def generate_answer(text, model):
    messages = [{"role": "system", "content": SYSTEM_PROMPT.format(categories='\n'.join(categories))}]
    prompt = "Текст: {text} \n" \
                "Ответ: "
    messages.append({"role": "user", "content": prompt.format(text=text)})
    answer = ""
    for part in model.create_chat_completion(
        messages,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repeat_penalty=repeat_penalty,
        stream=True,
    ):
        delta = part["choices"][0]["delta"]
        if "content" in delta:
            answer += delta["content"]
            print(delta["content"], end="", flush=True)
    return answer

In [6]:
def preparation_teg(teg):
  if teg.find(":") == -1:
    return teg.strip()
  return teg[:teg.find(":")].strip()

In [7]:
def determine_1_level_categories(x):
  result = x.split(",")
  result = list(map(lambda x: preparation_teg(x), result))
  return result

In [8]:
def iou_metric(ground_truth, predictions):
    iou =  len(set.intersection(set(ground_truth), set(predictions)))
    iou = iou/(len(set(ground_truth).union(set(predictions))))
    return iou

In [9]:
def split_tags(tag_list):
    final_tag_list = []
    for tag in tag_list:
        tags = tag.split(": ")
        if len(tags) == 3:
            final_tag_list.append(tags[0])
            final_tag_list.append(tags[0] + ": " + tags[1])
            final_tag_list.append(tags[0]+ ": " + tags[1] + ": " + tags[2])
        elif len(tags) == 2:
            final_tag_list.append(tags[0])
            final_tag_list.append(tags[0] + ": " + tags[1])
        elif len(tags) == 1:
            final_tag_list.append(tags[0])
        else:
            print("NOT IMPLEMENTED!!!!", tag)
    return final_tag_list

In [10]:
def find_iou_for_sample_submission(pred_submission, true_submission):
    ground_truth_df = true_submission
    ground_truth_df["tags"] = ground_truth_df["tags"].apply(lambda l: l.split(', '))
    ground_truth_df["tags_split"] = ground_truth_df["tags"].apply(lambda l: split_tags(l))

    predictions_df = pred_submission
    predictions_df["predicted_tags"] = predictions_df["predicted_tags"].apply(lambda l: l.split(', '))
    predictions_df["predicted_tags_split"] = predictions_df["predicted_tags"].apply(lambda l: split_tags(l))
    iou=0
    counter = 0
    for i, row in ground_truth_df.iterrows():
        predicted_tags = predictions_df[predictions_df["video_id"]==row["video_id"]]["predicted_tags_split"].values[0]
        iou_temp=iou_metric(row['tags_split'], predicted_tags)
        iou+=iou_temp
        counter+=1

    return iou/counter

In [43]:
def write_answer_to_csv(row):
    with open('data/pred_data.csv', 'a', newline='') as csvfile:
        csv.writer(csvfile).writerow(row) 

In [56]:
import time
import csv
from clear_text import clear_str
def generate_pred_submission(df):
    df_pred = pd.DataFrame(columns=("video_id","title","description","predicted_tags"))
    df_pred.to_csv('data/pred_data.csv')
    model = Llama(
        model_path="data/model-q4_K.gguf",
        n_ctx=n_ctx,
        n_parts=1,
        verbose=True,
    )
    list_pred = []
    for i, row in df.iterrows():
        text = clear_str(row["title"] + "." + row["description"])
        answer = generate_answer(text, model)
        row["predicted_tags"] = answer
        write_answer_to_csv(answer)
        list_pred.append(answer)
    pred_submission = df.copy()
    pred_submission["predicted_tags"] = list_pred
    return pred_submission
    

In [58]:
pred_submission = generate_pred_submission(df)

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from data/model-q4_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = saiga_llama3_8b
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.attention.head_count_kv u32        

Текст: тьяп март тьяп поздравлять лиза звезда шок утро тьяп начинаться недвусмысленный напоминание лиза подарок март который разумеется забыть признаваться это стать пообещать сюрприз готовый вечер тьяп обращаться помощь выбор подарок свой друг чернобыль это помочь понравиться сюрприз лиза также стас михайлов забыть балкон узнать посмотреть серия
Ответ: Семья и отношения, Массовая культура

llama_perf_context_print:        load time =   82335.63 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   719 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   136 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  137623.23 ms /   855 tokens
Llama.generate: 596 prefix-match hit, remaining 233 prompt tokens to eval


Массовая культура

llama_perf_context_print:        load time =   82335.63 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   233 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     6 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   29897.67 ms /   239 tokens
Llama.generate: 596 prefix-match hit, remaining 151 prompt tokens to eval


...

llama_perf_context_print:        load time =   82335.63 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    64 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    11 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   10265.05 ms /    75 tokens
Llama.generate: 596 prefix-match hit, remaining 114 prompt tokens to eval


Массовая культура

llama_perf_context_print:        load time =   82335.63 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   114 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     6 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   12860.04 ms /   120 tokens
Llama.generate: 596 prefix-match hit, remaining 106 prompt tokens to eval


Массовая культура

llama_perf_context_print:        load time =   82335.63 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   106 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     6 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   12054.26 ms /   112 tokens


In [61]:
pred_submission

,video_id,title,description,predicted_tags
0,c00d1b173ed8680c155b773d43af7791,Тяпа | #24 | 8 МАРТА. ТЯПА ПОЗДРАВЛЯЕТ ЛИЗУ. З...,Утро Тяпы начинается с недвусмысленного напоми...,Текст: тьяп март тьяп поздравлять лиза звезда ...
1,6064c4511a067f2e1d1cef81c21421e8,БЮРО ДОБРЫХ ДЕЛ №5 | ХУДОЖНИК ИЗ АСТРАХАНИ | К...,Добро пожаловать на канал “Бюро Добрых Дел”! Е...,Массовая культура
2,3071e2054a02a03150463e514f05fe90,Такой расклад. Эфир 17. Таро. Про личные качес...,В этом выпуске шоу «Такой расклад» вы увидите ...,"Медицина, Религия и духовность"
3,60867179613376c8b56f0705114a7685,TREXTRAVELER в ГрандТуре «Байкальская миля 202...,"Визит в Государственный природный заказник ""Су...",Путешествия
4,706f44ef4711eaee99035260c2d1ab86,Тру ДЕТЕКТОР I #5,Вы смотрите шоу с детектором лжи «Тру ДЕТЕКТОР...,Массовая культура
...,...,...,...,...
194,0f84219f46782fca9482fb88da146ed3,Диктант со звездой | Выпуск 7 | Чипинкос,В этом выпуске - рэпер Чипинкос останется один...,"Массовая культура, Музыка и аудио."
195,2f67917a65dd8d5b348de81d15b1d8ff,Полупановы. «Эпизод 16»,В этом эпизоде Игорь готовится к любимому праз...,"Массовая культура, Информационные технологии"
196,6fa39a883591bc0e2cf7ef6c5349c9df,"Салют, Начальник! 2 сезон, 6 серия",Назаров устраивает майору «сюрприз» прямо за с...,"Массовая культура, Семья и отношения"
197,bff493ac045715566a86cd7e84051938,Абьюз-шоу |Выпуск №7 часть 1I Алина Егорова,Героиней этого выпуска «Абьюз шоу» стала Алина...,Массовая культура


In [62]:
pred_submission.to_csv('data/pred_data2.csv', index=False, encoding='utf-8')

In [60]:
find_iou_for_sample_submission(pred_submission, part_df)


AttributeError: 'list' object has no attribute 'split'

In [19]:
df1

,video_id,title,description,tags,tags_split
121,01bd6b00e670655ca7ff484ff55d8ad9,ПО РУКАМ #8,Никита Юрковец и Низами Тагиев откроют этот вы...,"[Спорт, Массовая культура]","[Спорт, Массовая культура]"
821,acba788b86a3bd559044aadc488455c3,"Команда ""3/21"" в ГрандТуре-2022: заказник ""Леб...",Удивительная красота алтайской зимы не может о...,"[Путешествия, События и достопримечательности:...","[Путешествия, События и достопримечательности,..."
394,663ac16d23ef52bd261af0a41e6d8f6b,Роман Юнусов и блогер Кирилл Нечаев метают коп...,В новом выпуске шоу «Спортивный Интерес» Рома ...,"[Массовая культура, Спорт]","[Массовая культура, Спорт]"
364,a5ca18452c2ad26eeaa2007683840755,Пацанский клининг. Шоу «ЧистоТачка» | Белая бе...,Тяпу и Андрюху «Чернобыля» испугает своим сост...,[Массовая культура: Юмор и сатира],"[Массовая культура, Массовая культура: Юмор и ..."
459,8769ca8f5a3e0f53ff89c19e98392d76,"Мама 21 века| 2 сезон, 5 выпуск | Льготы для м...",Многодетным семьям в России оказывается поддер...,"[Семья и отношения, Личные финансы: Финансовая...","[Семья и отношения, Личные финансы, Личные фин..."
...,...,...,...,...,...
533,8884c062bfa7739bb0bd71bb02fd7893,Байкальская миля 2020: лед и скорость!,На фестивале скорости «Байкальская миля 2020»...,"[Транспорт, Спорт: Автогонки, События и достоп...","[Транспорт, Спорт, Спорт: Автогонки, События и..."
191,72ac02e9fa513e91af1c02dd2d79eb6a,НАЧАЛЬНИК ПОЦЕЛОВАЛ ОЛЮ ПДН?,Телеграм канал Начальника: Телеграм канал Оли...,[Массовая культура: Юмор и сатира],"[Массовая культура, Массовая культура: Юмор и ..."
311,54e99123e2326dc0b8d6c4f743da959c,Давай сделаем ЭТО на кухне | Найти любовь в Ин...,"Честно, хлёстко и порой на грани. Влад Чижов с...","[Еда и напитки: Кулинария, Массовая культура, ...","[Еда и напитки, Еда и напитки: Кулинария, Масс..."
747,0b73cea7b2dfdd7048dd84b95f8b9b0e,«МузЛофт - Подкаст» с Ириной Нельсон - 1 июля ...,"Подписывайся, чтобы не пропустить!","[Массовая культура, Музыка и аудио]","[Массовая культура, Музыка и аудио]"
